In [1]:
import { config } from "https://deno.land/x/dotenv/mod.ts";
import {z} from 'https://deno.land/x/zod@v3.23.8/mod.ts';
import { StructuredTool } from "https://esm.sh/@langchain/core/tools";
import { formatToOpenAITool } from "https://esm.sh/@langchain/openai";
import OpenAI from "https://deno.land/x/openai@v4.51.0/mod.ts";

const env = await config(); 

In [2]:
const OPENAI_API_KEY = env.OPENAI_API_KEY

In [3]:
import { ChatOpenAI } from "https://esm.sh/@langchain/openai";

In [4]:
const openai_client = new OpenAI({apiKey:OPENAI_API_KEY})

In [5]:
const client = new ChatOpenAI({apiKey:OPENAI_API_KEY})

In [6]:
await client.invoke("hi there")

g {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello! How can I assist you today?",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 9, promptTokens: 9, totalTokens: 18 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: [],
  usage_metadata: { input_tokens: 9, output_tokens: 9, total_tokens: 18 }
}

In [7]:
class AddNums extends StructuredTool {
    name = "add numbers";
    description = "add two numbers together";

    schema =  z.object({
        num_1: z.number().describe("the first number to add"),
        num_2: z.number().describe("the second number to add"),
    })

    constructor() {
        super();
    }

    _call(input: z.infer<typeof this['schema']>): Promise<string> {
        return Promise.resolve(JSON.stringify(input, null, 2));
    } 
}

In [8]:
const tool = new AddNums()

In [9]:
tool._call({num_1: 3, num_2: 4}).then(console.log)

{
  "num_1": 3,
  "num_2": 4
}


Promise { undefined }

In [10]:
tool.schema.describe()

ZodObject {
  _type: undefined,
  _output: undefined,
  _input: undefined,
  _def: {
    shape: [Function: shape],
    unknownKeys: "strip",
    catchall: ZodNever {
      _type: undefined,
      _output: undefined,
      _input: undefined,
      _def: { typeName: "ZodNever" },
      spa: [Function: bound safeParseAsync] AsyncFunction,
      parse: [Function: bound parse],
      safeParse: [Function: bound safeParse],
      parseAsync: [Function: bound parseAsync] AsyncFunction,
      safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
      refine: [Function: bound refine],
      refinement: [Function: bound refinement],
      superRefine: [Function: bound superRefine],
      optional: [Function: bound optional],
      nullable: [Function: bound nullable],
      nullish: [Function: bound nullish],
      array: [Function: bound array],
      promise: [Function: bound promise],
      or: [Function: bound or],
      and: [Function: bound and],
      transform: [Function: boun

In [11]:
typeof(3)

"number"

In [12]:
const addTool = formatToOpenAITool(new AddNums())

In [13]:
addTool

{
  type: "function",
  function: {
    name: "add numbers",
    description: "add two numbers together",
    parameters: {
      type: "object",
      properties: {
        num_1: { type: "number", description: "the first number to add" },
        num_2: { type: "number", description: "the second number to add" }
      },
      required: [ "num_1", "num_2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

In [14]:
await openai_client.chat.completions.create({
      messages: [{ role: 'user', content: "hello tell me a joke" }],
      model: 'gpt-3.5-turbo-0125',
    });

{
  id: "chatcmpl-9bVg0LZyhdw5F5uPKegRkbZX4iyZ4",
  object: "chat.completion",
  created: 1718726928,
  model: "gpt-3.5-turbo-0125",
  choices: [
    {
      index: 0,
      message: {
        role: "assistant",
        content: "Sure! Why couldn't the bicycle stand up by itself?\n" +
          "\n" +
          "Because it was two-tired!"
      },
      logprobs: null,
      finish_reason: "stop"
    }
  ],
  usage: { prompt_tokens: 12, completion_tokens: 19, total_tokens: 31 },
  system_fingerprint: null
}

In [15]:
addTool

{
  type: "function",
  function: {
    name: "add numbers",
    description: "add two numbers together",
    parameters: {
      type: "object",
      properties: {
        num_1: { type: "number", description: "the first number to add" },
        num_2: { type: "number", description: "the second number to add" }
      },
      required: [ "num_1", "num_2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

In [16]:
let tools = [addTool]

In [21]:
await openai_client.chat.completions.create({
    model: 'gpt-3.5-turbo-0125',
    messages: [{ role: 'user', content: "Please add the numbers 3 and 50000" }],
    functions: [addTool.function] 
    });

Error: 400 Invalid 'functions[0].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.

In [22]:
addTool.function

{
  name: "add numbers",
  description: "add two numbers together",
  parameters: {
    type: "object",
    properties: {
      num_1: { type: "number", description: "the first number to add" },
      num_2: { type: "number", description: "the second number to add" }
    },
    required: [ "num_1", "num_2" ],
    additionalProperties: false,
    "$schema": "http://json-schema.org/draft-07/schema#"
  }
}